In [1]:
import pandas as pd
import numpy as np

import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

from google.cloud import bigquery

In [2]:
transactions = pd.read_csv('transactions2_data.csv', index_col=0).reset_index(drop = True)
buildings = pd.read_csv('buildings_data.csv', index_col=0).reset_index(drop =True)
schools = pd.read_csv('schools.csv')
medianrent = pd.read_csv('rent.csv')

In [3]:
def make_title(df):
    string_columns = df.select_dtypes(include=['object'])
    df[string_columns.columns] = string_columns.map(str.title)
    return df.reset_index(drop = True)

In [4]:
transactions.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [5]:
schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [6]:
medianrent

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-
...,...,...,...,...
11272,2023-Q1,YISHUN,2-RM,-
11273,2023-Q1,YISHUN,3-RM,2500
11274,2023-Q1,YISHUN,4-RM,3000
11275,2023-Q1,YISHUN,5-RM,3200


In [7]:
medianrent.head()

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-


# Flat Type

In [8]:
medianrent['flat_type'].unique()

array(['1-RM', '2-RM', '3-RM', '4-RM', '5-RM', 'EXEC'], dtype=object)

In [9]:
transactions['flat_type'].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

In [10]:
def load_flat_type(transactions):
    flat_type = pd.DataFrame(transactions['flat_type'].unique(), columns = ['flat_type'])
    return flat_type

load_flat_type(transactions)

,flat_type
0,2 ROOM
1,3 ROOM
2,4 ROOM
3,5 ROOM
4,EXECUTIVE
5,1 ROOM
6,MULTI-GENERATION


In [11]:
def onemap_api(data):
    async def fetch(session, blk_no, street):
        search_val = f"{blk_no} {street}"
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

        async with session.get(url) as response:
            try:
                query = await response.json()
                query = query['results'][0]  
                postal = query['POSTAL']
                return postal
            except:
                print(f'API Query failed for {blk_no} , {street}')
                return None
            
    async def main():
        tasks = []
        async with aiohttp.ClientSession() as session:
            for index, row in data.iterrows():
                task = asyncio.create_task(fetch(session, row['blk_no'], row['street']))
                tasks.append(task)
            results = await asyncio.gather(*tasks)
        return results

    return asyncio.run(main())

# Buildings

In [12]:
buildings

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,33,50,0,0,0,0,0,0,0,0
12822,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,...,31,47,0,0,0,0,0,0,0,0
12823,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,...,0,0,0,0,0,0,0,0,0,0
12824,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,0,0,0,0,0,0,136,170,0,0


In [13]:
def transform_buildings(buildings):
    # Run OneMap API to get postal code
    buildings['postal_code'] = onemap_api(buildings)
    return buildings

buildings = transform_buildings(buildings)

API Query failed for 141A , SERANGOON NTH AVE 2
API Query failed for 151 , MEI LING ST
API Query failed for 151 , RIVERVALE CRES
API Query failed for 170 , HOUGANG AVE 1
API Query failed for 170 , BT BATOK WEST AVE 8
API Query failed for 296C , BT BATOK ST 22
API Query failed for 326 , UBI AVE 1
API Query failed for 354 , YISHUN RING RD
API Query failed for 4 , WOODLANDS ST 12
API Query failed for 5 , BANDA ST
API Query failed for 550 , HOUGANG ST 51
API Query failed for 62 , LOR 4 TOA PAYOH
API Query failed for 634C , TAMPINES NTH DR 2


In [14]:
def transform_flats_sold(buildings):
    # Extracting required columns
    new_dataframe = pd.DataFrame()
    new_dataframe['postal_code'] = buildings['postal_code']
    
    # Extracting flat types and their respective sold counts
    renamed_flat_types = {'1room_sold': '1 ROOM', '2room_sold': '2 ROOM', '3room_sold': '3 ROOM',
                          '4room_sold': '4 ROOM', '5room_sold' : '5 ROOM', 'exec_sold' : 'EXECUTIVE', 
                          'multigen_sold' : 'MULTI-GENERATION'}

    # Creating new columns for each flat type and their sold counts
    for flat_type, renamed_flat_type in renamed_flat_types.items():
        new_dataframe[renamed_flat_type] = buildings[flat_type]
    
    # Reshaping the DataFrame to have flat_type as a column
    new_dataframe = pd.melt(new_dataframe, id_vars=['postal_code'], value_vars=list(renamed_flat_types.values()),
                            var_name='flat_type', value_name='flats_sold')
    
    return new_dataframe

flats_sold = transform_flats_sold(buildings)


In [15]:
flats_sold

,postal_code,flat_type,flats_sold
0,190001,1 ROOM,0
1,460001,1 ROOM,0
2,080001,1 ROOM,0
3,461001,1 ROOM,0
4,500001,1 ROOM,0
...,...,...,...
89777,531998,MULTI-GENERATION,0
89778,532998,MULTI-GENERATION,0
89779,530999,MULTI-GENERATION,0
89780,531999,MULTI-GENERATION,0


# Town

In [16]:
town_mapping = {
        'AMK': 'ANG MO KIO', 'BB': 'BUKIT BATOK', 'BD': 'BEDOK', 'BH': 'BISHAN', 'BM': 'BUKIT MERAH',
        'BP': 'BUKIT PANJANG', 'BT': 'BUKIT TIMAH', 'CCK': 'CHOA CHU KANG', 'CL': 'CLEMENTI', 'CT': 'CENTRAL AREA',
        'GL': 'GEYLANG', 'HG': 'HOUGANG', 'JE': 'JURONG EAST', 'JW': 'JURONG WEST', 'KWN': 'KALLANG/WHAMPOA',
        'MP': 'MARINE PARADE', 'PG': 'PUNGGOL', 'PRC': 'PASIR RIS', 'QT': 'QUEENSTOWN', 'SB': 'SEMBAWANG',
        'SGN': 'SERANGOON', 'SK': 'SENGKANG', 'TAP': 'TAMPINES', 'TG': 'TENGAH', 'TP' : 'TOA PAYOH' ,
        'WL' : 'WOODLANDS' , 'YS' : 'YISHUN'
}

In [17]:
buildings['bldg_contract_town'].unique()

array(['KWN', 'BD', 'CT', 'PRC', 'BM', 'QT', 'GL', 'HG', 'SGN', 'TP',
       'MP', 'WL', 'PG', 'BT', 'TAP', 'CCK', 'AMK', 'BH', 'BB', 'SB',
       'CL', 'BP', 'JE', 'SK', 'YS', 'JW', 'TG'], dtype=object)

In [18]:
def load_town(town_mapping):
    return pd.DataFrame(list(town_mapping.values()), columns = ['town'])

load_town(town_mapping)

,town
0,ANG MO KIO
1,BUKIT BATOK
2,BEDOK
3,BISHAN
4,BUKIT MERAH
5,BUKIT PANJANG
6,BUKIT TIMAH
7,CHOA CHU KANG
8,CLEMENTI
9,CENTRAL AREA


In [19]:
def rename_buildings_town(data):
    data = data.copy(deep = True)

    data['bldg_contract_town'] = data['bldg_contract_town'].map(town_mapping)
    return data

rename_buildings_town(buildings)['bldg_contract_town']

0        KALLANG/WHAMPOA
1                  BEDOK
2           CENTRAL AREA
3                  BEDOK
4              PASIR RIS
              ...       
12821            HOUGANG
12822            HOUGANG
12823            HOUGANG
12824            HOUGANG
12825            HOUGANG
Name: bldg_contract_town, Length: 12826, dtype: object

# Transform buildings v2

In [21]:
def drop_buildings_columns(buildings):
    retained_cols = ['postal_code', 'blk_no', 'street', 'max_floor_lvl', 'year_completed', 'multistorey_carpark', 'precinct_pavilion', 'market_hawker']
    buildings_new = buildings[retained_cols]
    return buildings_new

drop_buildings_columns(buildings)

,postal_code,blk_no,street,max_floor_lvl,year_completed,multistorey_carpark,precinct_pavilion,market_hawker
0,190001,1,BEACH RD,16,1970,N,N,N
1,460001,1,BEDOK STH AVE 1,14,1975,N,N,N
2,080001,1,CANTONMENT RD,2,2010,N,N,N
3,461001,1,CHAI CHEE RD,15,1982,N,N,N
4,500001,1,CHANGI VILLAGE RD,4,1975,N,N,N
...,...,...,...,...,...,...,...,...
12821,531998,998A,BUANGKOK CRES,18,2018,N,N,N
12822,532998,998B,BUANGKOK CRES,17,2018,N,N,N
12823,530999,999,BUANGKOK CRES,2,2018,N,Y,N
12824,531999,999A,BUANGKOK CRES,18,2018,N,N,N


# Median Rent Prices

In [26]:
medianrent.head()

,quarter,town,flat_type,median_rent,year
0,Q2,ANG MO KIO,1-RM,na,2005
1,Q2,ANG MO KIO,2-RM,na,2005
2,Q2,ANG MO KIO,3-RM,800,2005
3,Q2,ANG MO KIO,4-RM,950,2005
4,Q2,ANG MO KIO,5-RM,-,2005


In [41]:
medianrent = pd.read_csv('rent.csv')

def transform_median_rent(rent):
    renamed_flat_types = {'1-RM': '1 ROOM', '2-RM': '2 ROOM', '3-RM': '3 ROOM',
                          '4-RM': '4 ROOM', '5-RM' : '5 ROOM', 'EXEC' : 'EXECUTIVE'}   
                          
    # Splitting quarter column into year and quarter columns
    rent[['year', 'quarter']] = rent['quarter'].str.split('-', expand=True)
    rent['year'] = pd.to_numeric(rent['year'])
    
    # remove unnecessary trailing spaces
    rent = rent.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # replace na values and - values with NaN
    rent.replace(r'^\s*na\s*$', np.nan, regex = True, inplace=True)
    rent.replace(r'^\s*-\s*$', np.nan, regex = True, inplace=True)
    
    # convert flat type to consistent format
    rent['flat_type'] = rent['flat_type'].map(renamed_flat_types)

    # reorder the columns
    rent = rent[['year', 'quarter', 'town', 'flat_type', 'median_rent']]
    return rent

transform_median_rent(medianrent)

,year,quarter,town,flat_type,median_rent
0,2005,Q2,ANG MO KIO,1 ROOM,NaN
1,2005,Q2,ANG MO KIO,2 ROOM,NaN
2,2005,Q2,ANG MO KIO,3 ROOM,800
3,2005,Q2,ANG MO KIO,4 ROOM,950
4,2005,Q2,ANG MO KIO,5 ROOM,NaN
...,...,...,...,...,...
11272,2023,Q1,YISHUN,2 ROOM,NaN
11273,2023,Q1,YISHUN,3 ROOM,2500
11274,2023,Q1,YISHUN,4 ROOM,3000
11275,2023,Q1,YISHUN,5 ROOM,3200


In [24]:
medianrent['flat_type'].unique()

array(['1-RM', '2-RM', '3-RM', '4-RM', '5-RM', 'EXEC'], dtype=object)

# Schools

In [42]:
schools = pd.read_csv('schools.csv')

In [45]:
schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [46]:
schools['mrt_desc'].unique()

array(['Admiralty Station', 'ADMIRALTY MRT', 'Yishun',
       'CANBERRA MRT, YISHUN MRT', 'Bishan MRT',
       'Redhill Station  Tiong Bahru Station',
       'MRT : NE16-Sengkang LRT : SW7-TongKang',
       'Yio Chu Kang MRT Station', 'ANG MO KIO MRT, YIO CHU KANG MRT',
       'Ang Mo Kio MRT Station', 'ANG MO KIO MRT', 'TANAH MERAH MRT',
       'Buona Vista', 'NEWTON MRT, STEVENS MRT',
       'DOVER MRT, BUONA VISTA MRT, ONE-NORTH MRT', 'Newton MRT Station',
       'Tampines and Simei Stations', 'CASHEW MRT',
       'DOWNTOWN LINE 2, CASHEW MRT', 'BARTLEY MRT, TAI SENG MRT',
       'Bangkit LRT, Fajar LRT Station', 'BRADDELL MRT, TOA PAYOH MRT',
       'Bedok MRT Station, Tanah Merah MRT Station',
       'BEDOK MRT, BEDOK RESERVOIR MRT', 'Boon Keng, Kallang',
       'BOON KENG MRT', 'Telok Blangah, Tiong Bahru & Redhill',
       'Lakeside & Boon Lay', 'BOON LAY MRT',
       'HOUGANG MRT, SERANGOON MRT, SENGKANG MRT', 'DAKOTA MRT',
       'BUKIT BATOK MRT', 'REDHILL MRT', 'CHOA CHU KAN

In [59]:
schools = pd.read_csv('schools.csv')
import re

def transform_schools(schools):
    def clean_and_format_town_name(text):
        # Remove stopwords and make format better
        text = text.title()
        cleaned_text = re.sub(r'\b(?:Station|Mrt|Lrt|Line|\([^)]*\))\b', '', text)
        return cleaned_text.strip()
    
    schools['mrt_desc'] = schools['mrt_desc'].apply(lambda x: clean_and_format_town_name(x))

    retained_cols = ['school_name', 'postal_code', 'mrt_desc', 'mainlevel_code', 
                     'nature_code', 'type_code', 'sap_ind', 'autonomous_ind', 'gifted_ind', 'ip_ind']
    
    return schools[retained_cols].rename(columns={'mrt_desc': 'closest_mrt'})

transform_schools(schools)

,school_name,postal_code,closest_mrt,mainlevel_code,nature_code,type_code,sap_ind,autonomous_ind,gifted_ind,ip_ind
0,ADMIRALTY PRIMARY SCHOOL,738907,Admiralty,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
1,ADMIRALTY SECONDARY SCHOOL,737916,Admiralty,SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
2,AHMAD IBRAHIM PRIMARY SCHOOL,768643,Yishun,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
3,AHMAD IBRAHIM SECONDARY SCHOOL,768928,"Canberra , Yishun",SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
4,AI TONG SCHOOL,579646,Bishan,PRIMARY,CO-ED SCHOOL,GOVERNMENT-AIDED SCH,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,169485,Tiong Bahru,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
342,ZHENGHUA PRIMARY SCHOOL,679002,: Fajar,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
343,ZHENGHUA SECONDARY SCHOOL,677741,Jelapang,SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
344,ZHONGHUA PRIMARY SCHOOL,556095,"Ang Mo Kio , Bishan , Serangoon",PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No


# Transactions

In [64]:
transactions = pd.read_csv('transactions2_data.csv', index_col=0).reset_index(drop = True)
transactions

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
175836,2024-03,YISHUN,EXECUTIVE,392,YISHUN AVE 6,07 TO 09,146.0,Maisonette,1988,63 years 04 months,830000.0
175837,2024-03,YISHUN,EXECUTIVE,406,YISHUN AVE 6,10 TO 12,142.0,Apartment,1988,63 years 06 months,815000.0
175838,2024-03,YISHUN,EXECUTIVE,326,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,63 years 05 months,820000.0
175839,2024-03,YISHUN,EXECUTIVE,360,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,63 years 05 months,860000.0
